## Solvating protein systems
Solvating a protein is not as simple as placing water molecules randomly around the protein. The water model used can significantly impact the accuracy and computational cost of the simulation. The TIP3P model is one of the most commonly used water models in molecular dynamics simulations. It's a simple and computationally efficient model, where each water molecule is represented as a rigid structure with three interaction sites: two positive hydrogen atoms and one negative oxygen atom. In OpenMM, you can solvate a protein with TIP3P water molecules using the Modeller.addSolvent() function.

However, the TIP3P model has its limitations. It is a rigid model, meaning that it does not account for the flexibility of the water molecules. This can lead to inaccuracies in the simulation, particularly when modeling specific types of interactions, such as hydrogen bonding. Moreover, TIP3P does not include polarization effects, which can be important in certain biological systems.

An alternative to explicit solvation (like with TIP3P) is implicit solvation, where the solvent is not represented by individual molecules, but rather by a continuous medium. This is implemented in OpenMM through Generalized Born (GB) models. Implicit solvation drastically reduces the number of particles in the system, leading to significantly faster simulations. It's particularly useful for simulations where solvent-solute interactions are important, but the fine details of the solvent structure are not.

However, implicit solvation also has its drawbacks. It is less accurate than explicit solvation for modeling the structure and dynamics of the solvation shell around the protein. It also fails to capture certain phenomena, such as the formation of solvent voids or cavities, and it cannot be used to simulate processes that involve explicit water molecules, such as the diffusion of small molecules through water.

TIP3P is a computationally efficient model that can provide a good approximation of water for many systems, but it has its limitations. Implicit solvation can speed up your simulations and may be a good choice for certain types of systems, but it lacks the accuracy of explicit solvation for modeling detailed solvent structure and dynamics.

## When explicit solvation is useful

Explicit solvent models are particularly important in a number of contexts where the specific arrangement or behavior of water molecules plays a critical role in the biological system being studied. Here are a few examples:

**Protein folding and stability:** Protein folding is driven by a delicate balance of various forces, including hydrophobic collapse, hydrogen bonding, and electrostatic interactions. Water plays a vital role in all of these, and its explicit representation allows for an accurate simulation of the folding process and stability.

**Enzyme catalysis:** Water often plays a direct role in the mechanisms of enzymatic reactions, for example, as a reactant, a product, or through mediating proton transfers. In these cases, an explicit water model is necessary to accurately simulate the catalytic process.

**Protein-ligand interactions:** In drug design, the binding of a small molecule ligand to a protein can be significantly influenced by water. Water molecules can mediate protein-ligand interactions, and displacement of water from the binding site can contribute to the binding energy. Explicit water models can capture these effects.

**Protonation state effects:** The protonation state of a protein, which can be influenced by the surrounding water environment, can dramatically affect its structure and function. Explicit solvent simulations can help to capture these effects.

**Membrane proteins:** The interaction of membrane proteins with the lipid bilayer and the surrounding aqueous environment is critical for their function. Explicit water is necessary to correctly model these interactions.eractions.ynamics.

## Finding the ligand

PDB have two main record types atoms and hetatoms. Any ligand should be a hetatom if a pdb has been set up correctly.

There are two main approaches to find the ligand ID, 

1) We could open the pdb and examine it.
2) We can try and extract it from the PDB automatically with the bellow script.


In [7]:
from Bio.PDB import PDBParser

input_pdb_file = "assets/cookbook/pdbs_not_for_repo/6HAZ.pdb"  # Replace with the name of your PDB file

# Parse the PDB file and create a new structure
parser = PDBParser()
structure = parser.get_structure("protein", input_pdb_file)

ligand_residues = set()

for model in structure:
    for chain in model:
        for residue in chain:
            residue_id = residue.get_id()
            residue_name = residue.get_resname()
            if residue_id[0].startswith("H_"):  # Check if the residue is a hetero-residue
                if residue_name in ["ZN", "NA", "CL"]:
                    continue  # Skip ions
                ligand_residues.add(residue_name)

# if len(ligand_residues) == 1:  
ligand = list(ligand_residues) #[0]
print("ligand found: ", ligand)
ligand_residue_names = ligand


ligand found:  ['FX5']


/opt/conda/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8021.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 8098.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8175.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 8347.
  warnings.warn(


In [40]:
from Bio.PDB import *
import os

# Path to the folder where the modified PDBs will be saved
pdb_folder = 'assets/cookbook/pdbs_not_for_repo'  # Adjust as needed

# Function to remove selected chains and save new PDB
def remove_chains_and_save(pdb_file, chains_to_remove):
    global input_pdb_file

    # Convert input chain IDs to uppercase
    chains_to_remove = [chain.upper() for chain in chains_to_remove]

    parser = PDBParser()
    structure = parser.get_structure("structure", pdb_file)

    # Gather existing chain IDs in the structure
    existing_chain_ids = {chain.id for model in structure for chain in model.get_chains()}

    # Check if the chains to remove are in the existing chain IDs
    if not set(chains_to_remove).issubset(existing_chain_ids):
        print("Warning: One or more input chain IDs do not match existing chain IDs in the structure.")
        return

    for model in structure:
        for chain in list(model.get_chains()):
            if chain.id in chains_to_remove:
                model.detach_child(chain.id)

    io = PDBIO()
    io.set_structure(structure)
    new_pdb_file = os.path.join(pdb_folder, f"modified_{os.path.basename(pdb_file)}")
    io.save(new_pdb_file)
    input_pdb_file = new_pdb_file
    print(f"Modified PDB saved as {new_pdb_file}")

# UI components for selecting PDB file and chains
pdb_files = [f for f in os.listdir(pdb_folder) if f.endswith('.pdb')]

pdb_dropdown = widgets.Dropdown(options=pdb_files, description='PDB File:')
chains_input = widgets.Text(description='Chains to Remove:', placeholder='A,B,C')

def on_modify_button_clicked(b):
    with modification_output:
        modification_output.clear_output()
        pdb_file = os.path.join(pdb_folder, pdb_dropdown.value)
        chains_to_remove = chains_input.value.split(',')
        remove_chains_and_save(pdb_file, chains_to_remove)
         

modify_button = widgets.Button(description="Modify PDB")
modify_button.on_click(on_modify_button_clicked)

modification_output = widgets.Output()

display(pdb_dropdown, chains_input, modify_button, modification_output)
print(input_pdb_file)

Dropdown(description='PDB File:', options=('6HAZ.pdb', '6HAZ_A_chain_ligand_waters.pdb', 'modified_6HAZ.pdb'),…

Text(value='', description='Chains to Remove:', placeholder='A,B,C')

Button(description='Modify PDB', style=ButtonStyle())

Output()

assets/cookbook/pdbs_not_for_repo/modified_6HAZ.pdb


In [134]:
import nglview as nv
view = nv.show_structure_file("assets/cookbook/pdbs_not_for_repo/modified_6HAZ.pdb")
#view.add_ball_and_stick("protien") 
view

NGLWidget()

### Extracting the ligands

The next cell code extracts ligands from a protein structure and saves them as separate files. It does this by reading a PDB file, which is a common file format for protein structures.

1) The code defines the folder where the ligand files will be saved, called "assets/cookbook/Ligands".

2) The code uses a function called extract_ligand_residues to identify the ligand residues in the protein structure. These residues are the small molecules that the code is interested in.

3) The code creates a new folder for the ligand files and saves each ligand residue as a separate file in this folder. The ligand files are saved in two different formats, PDB and SDF, which are commonly used formats for small molecules.

4) The code uses a module called RDKit to convert the PDB files to SDF format. This is done so that the ligand files can be used in other software programs that require the SDF format.



In [35]:
import os
import shutil
from Bio.PDB import PDBParser, PDBIO, Select
from rdkit import Chem

# Set the input and output file names
ligands_subfolder = "assets/cookbook/Ligands"

print("using the file", input_pdb_file, "as the starting structure")

# Function to extract ligand residues
def extract_ligand_residues(chain, residue_names):
    return [residue for residue in chain if residue.get_resname() in residue_names]

# Clean up the ligands subfolder and create it again
if os.path.exists(ligands_subfolder):
    shutil.rmtree(ligands_subfolder)
os.makedirs(ligands_subfolder)

# Parse the PDB file and create a new structure
parser = PDBParser()
structure = parser.get_structure("protein", input_pdb_file)

# Iterate over chains and extract ligand residues using the function
ligand_residues = []
for chain in structure.get_chains():
    ligand_residues.extend(extract_ligand_residues(chain, ligand_residue_names))

#print(ligand_residues)
#for residue in ligand_residues: 
#    print(residue, residue.get_resname())

# Save each ligand residue to a separate PDB/SDF file
ligand_counts = {}
ligand_residue_names_split=[]

io = PDBIO()
for residue in ligand_residues:
    residue_name = residue.get_resname()

    # If the ligand has already been processed before, increase its count, otherwise set its count to 1
    ligand_counts[residue_name] = ligand_counts.get(residue_name, 0) + 1
    unique_identifier = ligand_counts[residue_name]

    # Modify the output filenames to include the unique identifier
    output_ligand_pdb_file = os.path.join(ligands_subfolder, f"{residue_name}_{unique_identifier}_ligand.pdb")
    output_ligand_sdf_file = os.path.join(ligands_subfolder, f"{residue_name}_{unique_identifier}_ligand.sdf")


    io.set_structure(residue)
    io.save(output_ligand_pdb_file)
    
    # Convert the PDB file to SDF
    with open(output_ligand_pdb_file, "r") as f:
        pdb_block = f.read()

    mol = Chem.MolFromPDBBlock(pdb_block)
    Chem.MolToMolFile(mol, output_ligand_sdf_file)
    print("Writing ligand file", output_ligand_pdb_file)
    print("Writing ligand file", output_ligand_sdf_file)
    ligand_residue_names_split.append(f"{residue_name}_{unique_identifier}")

using the file assets/cookbook/pdbs_not_for_repo/modified_6HAZ.pdb as the starting structure
Writing ligand file assets/cookbook/Ligands/FX5_1_ligand.pdb
Writing ligand file assets/cookbook/Ligands/FX5_1_ligand.sdf


## Visualising the ligand

The cell bellow lets you steps through all the possible ligands found in the origonal PDB and visualise them both using RDKIT and ngview. Select an entry from the dropdown and you'll be able to visualise it. 

The bellow code will auto add hydrogens, but take note that its usually a bad guess

In [36]:
import os
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, AllChem 
from openff.toolkit.topology import Molecule
import nglview as nv
import ipywidgets as widgets
from IPython.display import display

def visualize_ligand(ligand_file):
    rdkitmol = Chem.MolFromMolFile(ligand_file)
    # Add hydrogens only if the number of atoms is greater than 1
    if rdkitmol.GetNumAtoms() > 1:
        rdkitmol = Chem.AddHs(rdkitmol, addCoords=True)
    rdkitmol.UpdatePropertyCache(strict=False)

    # Assign stereochemistry
    Chem.AssignAtomChiralTagsFromStructure(rdkitmol)
    Chem.AssignStereochemistry(rdkitmol, force=True, cleanIt=True)
    Chem.AssignStereochemistryFrom3D(rdkitmol, replaceExistingTags=True)

    ligand = Molecule(rdkitmol, allow_undefined_stereo=True)

    # Draw shallow copy of 2d molecule for inspection 
    for mol in [rdkitmol.__copy__()]:
        AllChem.Compute2DCoords(mol)
        img = Chem.Draw.MolToImage(mol)  

    # Create NGLview visualization
    view = nv.NGLWidget()
    view.add_component(rdkitmol)
    view.representations = [{"type": "ball+stick", "params": {"multipleBond": "offset"}}]

    # Display RDKit image and NGLview side by side
    hbox = widgets.HBox([widgets.Image(value=img._repr_png_()), view])
    return hbox

def on_ligand_selected(change):
    selected_file = os.path.join('assets/cookbook/Ligands', change['new'])
    with visualization_output:
        visualization_output.clear_output()
        display(visualize_ligand(selected_file))


ligands_folder = 'assets/cookbook/Ligands'
ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('.sdf')]

dropdown = widgets.Dropdown(options=ligand_files,description='Ligand:',value=ligand_files[0])


dropdown.observe(on_ligand_selected, names='value')
visualization_output = widgets.Output()
display(dropdown)
display(visualization_output)

# Show default ligand view if there is only one ligand
if ligand_files[0] is not None:
    dropdown.value = ligand_files[0]
    on_ligand_selected({'new': ligand_files[0]})

Dropdown(description='Ligand:', options=('FX5_1_ligand.sdf',), value='FX5_1_ligand.sdf')

Output()

# OPTIONAL: Using Antechamber to assign better hyrodgenation of a Ligand

You might notice a red box about complaining about the chirality of centres around the ligand. Or that the number of hydrogens added is wrong for the ligand. Lets use antechamber to fix this.

Antechamber is a part of the AmberTools suite, specifically designed to prepare ligands for molecular simulations using Amber force fields. In this tutorial, we will discuss how to use Antechamber to assign a topology to a ligand by generating a mol2 file with AM1-BCC charges and GAFF atom types.

In [115]:
import subprocess
#from openbabel import pybel
import os


ligands_folder = 'assets/cookbook/Ligands'
ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('.pdb') and not f.endswith('_h.pdb')]
print(ligand_files)

for file in ligand_files: 
    # Get the input file's base name without the extension
    file_base_name, _ = os.path.splitext(file)

    # Add hydrogens to the input PDB file using the 'reduce' program
    output_pdb_with_h = os.path.join(ligands_folder, f"{file_base_name}_with_h.pdb")
    with open(output_pdb_with_h, "w") as outfile:
        subprocess.run(["reduce", "-BUILD", os.path.join(ligands_folder, file)], check=True, text=True, stdout=outfile)


['FX5_1_ligand.pdb']


Processing file: "assets/cookbook/Ligands/FX5_1_ligand.pdb"
Building His ring NH Hydrogens.
Building or keeping OH & SH Hydrogens.
Rotating existing OH & SH Hydrogens
VDW dot density = 16/A^2
Probe radius = 0.25A
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Flipping Asn, Gln and His groups.
For each flip state, bumps where gap is more than 0.4A are indicated with '!'.
Rotating NH3 Hydrogens.
Not processing Met methyls.
 Singles(size 1): B1501 FX5 O13 
 orientation 3: B1501 FX5 O13 :   rot   19: bump=0.000, HB=0.000, total=0.062
Found 18 hydrogens (18 hets)
Standardized 18 hydrogens (18 hets)
Added 0 hydrogens (0 hets)
Adjusted 1 group(s)
If you publish work which uses reduce, please cite:
Word, et. al. (1999) J. Mol. Biol. 285, 1735-1747.
For more information see http://kinemage.biochem.duke.edu


In [33]:
# import os
# from simtk.openmm import Vec3
# from simtk.openmm.app import Modeller, ForceField, PDBFile
# from openmmforcefields.generators import GAFFTemplateGenerator
# from openff.toolkit.topology import Molecule, Topology
# from simtk import unit

# ligands_folder = 'assets/cookbook/Ligands'
# ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('.sdf')]

# for file in ligand_files:
#     file_base_name, _ = os.path.splitext(file)
#     input_file_path = os.path.join(ligands_folder, file)

#     # Create OpenFF Molecule from SDF
#     openff_mol = Molecule.from_file(input_file_path, allow_undefined_stereo=True)

#     # Convert OpenFF Molecule to OpenFF Topology
#     openff_topology = Topology.from_molecules([openff_mol])

#     # Convert OpenFF Topology to OpenMM Topology
#     openmm_topology = openff_topology.to_openmm()

#     # Get the OpenFF Molecule conformer positions and convert to OpenMM Vec3 with proper units
#     openmm_positions = [Vec3(*pos) for pos in openff_mol.conformers[0].value_in_unit(unit.nanometer)]

#     # Multiply by unit.nanometer to convert to OpenMM Quantity
#     openmm_positions = [pos * unit.nanometer for pos in openmm_positions]

#     modeller = Modeller(openmm_topology, openmm_positions)

#     # Create a GAFFTemplateGenerator instance
#     gaff_generator = GAFFTemplateGenerator(molecules=[openff_mol])

#     forcefield = ForceField()
#     forcefield.registerTemplateGenerator(gaff_generator.generator)

#     modeller.addHydrogens(forcefield)

#     output_pdb_with_h = os.path.join(ligands_folder, f"{file_base_name}_with_h.pdb")
#     with open(output_pdb_with_h, 'w') as outfile:
#         PDBFile.writeFile(modeller.topology, modeller.positions, outfile)


## Reprocess the ligand with openbabel into an sdf file

we can use openbabel to reprocess the rdkit generated connectivity and potnetially fix topology issues for when chiral mallassignement arrises. In this case we are using it to put it into a sdf file format

In [118]:
from openbabel import pybel
from rdkit import Chem
import nglview


ligands_folder = 'assets/cookbook/Ligands'
ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('_h.pdb')]

for input_file in ligand_files :
    print(input_file)
    file_base_name, _ = os.path.splitext(file)
    mol = next(pybel.readfile("pdb", ligands_folder+"/"+input_file))
    #mol.addh()
    mol.write("sdf", ligands_folder+"/"+file_base_name+"with_h.sdf", overwrite=True)
    


FX5_1_ligand_with_h.pdb


## Experimental reprocessing from pdb to sdf

This code block performs the conversion of ligand structures from PDB to SDF format, and subsequently processes these structures for chemical analysis. Initially, Open Babel (pybel) is utilized to accurately convert PDB files into SDF files, ensuring correct inference and retention of aromaticity, a crucial aspect often lost in straightforward conversions. Following this, RDKit is employed to read the generated SDF files. Within this RDKit processing step, specific attention is given to nitrogen atoms with four bonds, which are adjusted to reflect a quaternary ammonium ion's positive charge state. After this adjustment, RDKit's sanitization process is applied to each molecule to perceive aromaticity and establish other molecular properties accurately. Finally, the processed molecular structures are written back into SDF format, making them suitable for further computational chemistry analyses or molecular modeling tasks.

In [139]:
from openbabel import pybel
from rdkit import Chem
import os

ligands_folder = 'assets/cookbook/Ligands'
ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('_h.pdb')]

for input_file in ligand_files:
    file_base_name, _ = os.path.splitext(input_file)

    # Convert PDB files to SDF using Open Babel
    mol = next(pybel.readfile("pdb", os.path.join(ligands_folder, input_file)))
    sdf_output_file = os.path.join(ligands_folder, file_base_name + "_with_h.sdf")
    mol.write("sdf", sdf_output_file, overwrite=True)

    # Read the SDF file with RDKit for further processing
    rdkit_mol = Chem.SDMolSupplier(sdf_output_file, removeHs=False, sanitize=False)[0]
    if rdkit_mol is None:
        print(f"Could not load {sdf_output_file} with RDKit.")
        continue

    # Adjust the nitrogen atom's charge state if necessary and warn
    for atom in rdkit_mol.GetAtoms():
        if atom.GetAtomicNum() == 7 and atom.GetTotalDegree() == 4:  # Nitrogen with 4 bonds
            atom_info = f"Atom Index: {atom.GetIdx()}, Atom Type: {atom.GetSymbol()}, Connected Atoms: "
            connected_atoms = [rdkit_mol.GetAtomWithIdx(b.GetOtherAtomIdx(atom.GetIdx())).GetSymbol() for b in atom.GetBonds()]
            atom_info += ', '.join(connected_atoms)
            print("Warning: found possible quaternary ammonium ion at", atom_info)            
            atom.SetFormalCharge(1)  # Assuming a positive charge for quaternary ammonium

    # Sanitize the molecule to perceive aromaticity and other properties
    Chem.SanitizeMol(rdkit_mol, Chem.SanitizeFlags.SANITIZE_ALL)

    # Write the processed molecule back to SDF
    writer = Chem.SDWriter(sdf_output_file)
    writer.write(rdkit_mol)
    writer.close()

### Veiwing the reprotonated ligand

In [123]:
import os
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, AllChem 
from openff.toolkit.topology import Molecule
import nglview as nv
import ipywidgets as widgets
from IPython.display import display

def visualize_ligand(ligand_file):
    # Need to use suplier when there are hydrogens
    mol_supplier = Chem.SDMolSupplier(ligand_file, removeHs=False, sanitize=False)
    rdkitmol = mol_supplier[0] if mol_supplier[0] is not None else None   # rdkitmol = mol_supplier[0]
    with open(ligand_file, "r") as f:
         sdf_data = f.read()
    
    # Draw shallow copy of 2d molecule for inspection 
    for mol in [rdkitmol.__copy__()]:
        AllChem.Compute2DCoords(mol)
        img = Chem.Draw.MolToImage(mol)  

    # Create NGLview visualization
    view = nv.NGLWidget()
    view.add_component(rdkitmol)
    view.representations = [{"type": "ball+stick", "params": {"multipleBond": "offset"}, "showH": "true"}]

    # Display RDKit image and NGLview side by side
    hbox = widgets.HBox([widgets.Image(value=img._repr_png_()), view])
    return hbox

def on_ligand_selected(change):
    selected_file = os.path.join('assets/cookbook/Ligands', change['new'])
    with visualization_output:
        visualization_output.clear_output()
        display(visualize_ligand(selected_file))


ligands_folder = 'assets/cookbook/Ligands'
ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('with_h.sdf')]
print(ligand_files)
dropdown = widgets.Dropdown(options=ligand_files,description='Ligand:',value=ligand_files[0])


dropdown.observe(on_ligand_selected, names='value')
visualization_output = widgets.Output()
display(dropdown)
display(visualization_output)

# Show default ligand view if there is only one ligand
if ligand_files[0] is not None:
    dropdown.value = ligand_files[0]
    on_ligand_selected({'new': ligand_files[0]})

['FX5_1_ligand_with_h_with_h.sdf', 'FX5_1_ligandwith_h.sdf']


Dropdown(description='Ligand:', options=('FX5_1_ligand_with_h_with_h.sdf', 'FX5_1_ligandwith_h.sdf'), value='F…

Output()

# Creating OpenMM Compatible Files Using RDKit
## Introduction
In this section, we will be building protein-ligand complexes by creating an OpenMM-compatible ligand file (sdf/mol2) from an RDKit molecule. The ligand we will be using is in .sdf file format that we will need to convert it and prepear it for simulation. To ensure smooth simulation need to ensure that its desired position is relative to the protein to avoid it being too far away or inside the protein. Nanome can be used to design the ligand and construct a good starting positions for dynamics, which can significantly accelerate MD.

## Loading the Ligand
To begin, we will load the ligand from the .sdf file format. Once we have the ligand, we can add hydrogens to it using RDKit. 

## Converting to OpenMM-Compatible Format
After adding hydrogens to the ligand, we can convert it to an OpenMM-compatible format using the OpenMM.Molecule function. This will allow us to build the protein-ligand complex using OpenMM by getting them both in the same format.

We will inspact with both RdKits 2D viewer and ngview with bond orders

 ## Converting an RDkit molecule to an OpenMM molecule
 
To add the ligand back to the system, we will use a pre-split ligand file and convert it from SDF format into an OpenFF format and then into an OpenMM format. The below function will do the heavy lifting, for this example, we do not need to understand the deeper process, but effectively it works out the elements in the SDF file and processes it in order to construct a topology (connectivity)  for the molecule, this can then be used to used the general amber forcefield (GaFF) to create all the parameters for the bond, angle and dihedrals for the ligand.

In [124]:
import openmm.app as app
from openff.toolkit.topology import Molecule

def rdkit_to_openmm(rdkit_mol, name="LIG"):
    """
    Convert an RDKit molecule to an OpenMM molecule.

    Parameters
    ----------
    rdkit_mol: rdkit.Chem.rdchem.Mol
        RDKit molecule to convert.
    name: str
        Molecule name.

    Returns
    -------
    omm_molecule: simtk.openmm.app.Modeller
        OpenMM modeller object holding the molecule of interest.
    """
    # convert RDKit to OpenFF
    off_mol = Molecule.from_rdkit(rdkit_mol, allow_undefined_stereo=True)

    # add name for molecule
    off_mol.name = name

    # add names for atoms
    element_counter_dict = {}
    for off_atom, rdkit_atom in zip(off_mol.atoms, rdkit_mol.GetAtoms()):
        element = rdkit_atom.GetSymbol()
        if element in element_counter_dict.keys():
            element_counter_dict[element] += 1
        else:
            element_counter_dict[element] = 1
        off_atom.name = element + str(element_counter_dict[element])

    # convert from OpenFF to OpenMM
    off_mol_topology = off_mol.to_topology()
    mol_topology = off_mol_topology.to_openmm()
    mol_positions = off_mol.conformers[0]

    # convert units from Ångström to nanometers
    # since OpenMM works in nm
    mol_positions = mol_positions.to("nanometers")

    # combine topology and positions in modeller object
    omm_mol = app.Modeller(mol_topology, mol_positions)

    return omm_mol, off_mol

### Create openMM objects from the hydrated ligand sdf file

In [76]:
from rdkit import Chem
from rdkit.Chem import AllChem
import os

ligands_folder = 'assets/cookbook/Ligands'
ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('_h.pdb')]

for input_file in ligand_files:
    file_base_name, _ = os.path.splitext(input_file)
    
    # Read PDB file with RDKit
    pdb_file_path = os.path.join(ligands_folder, input_file)
    rdkit_mol = Chem.MolFromPDBFile(pdb_file_path, removeHs=False, sanitize=False)

    if rdkit_mol is None:
        print(f"Could not load {pdb_file_path} with RDKit.")
        continue

    # Adjust the nitrogen atom's charge state if necessary
    for atom in rdkit_mol.GetAtoms():
        if atom.GetAtomicNum() == 7 and atom.GetTotalDegree() == 4:  # Nitrogen with 4 bonds
            atom.SetFormalCharge(1)  # Assuming a positive charge for quaternary ammonium

    # Write to SDF
    sdf_output_file = os.path.join(ligands_folder, file_base_name + "_with_h.sdf")
    writer = Chem.SDWriter(sdf_output_file)
    writer.write(rdkit_mol)
    writer.close()


In [125]:
from rdkit import Chem
import os

# Load the hydrated ligand files
ligands_folder = 'assets/cookbook/Ligands'
ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('_h.sdf')]

# loop through each ligand file found
for file in ligand_files:
    rdkit_ligand = Chem.MolFromMolFile(ligands_folder+"/"+file, sanitize=False, removeHs=False)
    ligand_name= "LIG"
    omm_ligand, off_ligand = rdkit_to_openmm(rdkit_ligand, ligand_name)

In [61]:
# from rdkit import Chem
# import subprocess

# ligands_folder = 'assets/cookbook/Ligands'
# ligand_files = [f for f in os.listdir(ligands_folder) if f.endswith('_h.pdb')]

# for file in ligand_files: 
#     # Get the input file's base name without the extension
#     file_base_name, _ = os.path.splitext(file)

#     # Add hydrogens to the input PDB file using the 'reduce' program
#     output_sdf_with_h = os.path.join(ligands_folder, f"{file_base_name}.sdf")
    
#     mol = next(pybel.readfile("pdb", 'assets/cookbook/Ligands'+"/"+file))
#     #mol.addh()
#     mol.write("sdf", output_sdf_with_h, overwrite=True)

#     rdkit_ligand = Chem.MolFromMolFile(output_sdf_with_h)
#     print(file)
#     ligand_name= "LIG"
#     omm_ligand = rdkit_to_openmm(rdkit_ligand, ligand_name)

# Viewing the prepeared protein

In [101]:
from openmm.app import * 
from openmm import *
from openmm.unit import *
from openmm.openmm import *
from pdbfixer import PDBFixer
import subprocess

# PDB file that we will use as a starting structure
pdb_start = "assets/cookbook/pdbs_not_for_repo/modified_6HAZ.pdb"

# PDB file that we will use as the cleaned output structure
pdb_out = 'assets/cookbook/cleaned_output.pdb'

#output_capture = io.StringIO()

# Use amber4pdb to clean up records for use with amber forcefield
pdb4amber_result = subprocess.run(["pdb4amber", "--nohyd", "--dry", pdb_start],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

with open(pdb_out, 'wb') as f:
    f.write(pdb4amber_result.stdout.encode("utf-8")) 

# Use reduce to add hydrogens according to ambers preferences    
#try:
#    out = subprocess.check_output(["reduce", "-build", "-nuclear", "assets/cookbook/cleaned_output.pdb"], stderr=subprocess.PIPE)
#except subprocess.CalledProcessError as e:
#    print("Error message from reduce:", e.stderr.decode())

# Use OpenMMs pdbfixer to fix some final issues that can crop up
fixed_pdb = PDBFixer(filename=pdb_out)
fixed_pdb.findMissingResidues()
fixed_pdb.findNonstandardResidues()
#fixer.replaceNonstandardResidues(
fixed_pdb.removeHeterogens(True) # comment to run with ligand
fixed_pdb.findMissingAtoms()
fixed_pdb.addMissingAtoms()
fixed_pdb.addMissingHydrogens(7.0)
PDBFile.writeFile(fixed_pdb.topology, fixed_pdb.positions, open(pdb_out, 'w'))



## View the uncomplexed protien

In [102]:
import nglview as nv
view = nv.show_structure_file("assets/cookbook/cleaned_output.pdb")
#view.add_ball_and_stick("protien") 
view

NGLWidget()

## Merge the protien and the ligand topologies

In [126]:
import mdtraj as md
import numpy as np
def merge_protein_and_ligand(protein, ligand):
    """
    Merge two OpenMM objects.

    Parameters
    ----------
    protein: pdbfixer.pdbfixer.PDBFixer
        Protein to merge.
    ligand: simtk.openmm.app.Modeller
        Ligand to merge.

    Returns
    -------
    complex_topology: simtk.openmm.app.topology.Topology
        The merged topology.
    complex_positions: simtk.unit.quantity.Quantity
        The merged positions.
    """
    # combine topologies
    md_protein_topology = md.Topology.from_openmm(protein.topology)  # using mdtraj for protein top
    md_ligand_topology = md.Topology.from_openmm(ligand.topology)  # using mdtraj for ligand top
    md_complex_topology = md_protein_topology.join(md_ligand_topology)  # add them together
    complex_topology = md_complex_topology.to_openmm()

    # combine positions
    total_atoms = len(protein.positions) + len(ligand.positions)

    # create an array for storing all atom positions as tupels containing a value and a unit
    # called OpenMM Quantities
    complex_positions = unit.Quantity(np.zeros([total_atoms, 3]), unit=unit.nanometers)
    complex_positions[: len(protein.positions)] = protein.positions  # add protein positions
    complex_positions[len(protein.positions) :] = ligand.positions  # add ligand positions

    return complex_topology, complex_positions

In [127]:
complex_topology, complex_positions = merge_protein_and_ligand(fixed_pdb, omm_ligand)
print("Complex topology has", complex_topology.getNumAtoms(), "atoms.")
complex_topology

Complex topology has 1935 atoms.


/opt/conda/lib/python3.10/site-packages/openmm/unit/quantity.py:753: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  self._value[key] = value / self.unit


<Topology; 2 chains, 114 residues, 1935 atoms, 1953 bonds>

# Viewing the prepearped protien ligand complex PDB

We can view the new protien ligand complex held in the "complex_topology/Complex_positions" by exporting to the temp.pdb

In [128]:
PDBFile.writeFile(complex_topology, complex_positions, open('assets/cookbook/temp.pdb', 'w'))
view = nv.show_structure_file("assets/cookbook/temp.pdb")
view

NGLWidget()

# Create the parameters for the ligand 

We have generated both the ligand positions and a topology combined with the protein (complex_positions and complex_topology). Our final setep is to register the ligand as part of the Forcefield using the Gaff Generator, which effectively parses and adds the physical description of the ligand to the standard protien Forcefield.

In [87]:
!python -m openmm.testInstallation


OpenMM Version: 8.1
Git Revision: 43f571d90fc1c882848cbd9b55e097a1775a9075

There are 3 Platforms available:

1 Reference - Successfully computed forces
2 CPU - Successfully computed forces
3 CUDA - Successfully computed forces

Median difference in forces between platforms:

Reference vs. CPU: 6.30914e-06
Reference vs. CUDA: 6.73363e-06
CPU vs. CUDA: 6.96327e-07

All differences are within tolerance.


In [129]:
from sys import stdout
from openmm.app import ForceField
from openff.toolkit.topology import Molecule, Topology
from openmmforcefields.generators import GAFFTemplateGenerator
from mdtraj.reporters import XTCReporter

# The forcefieild for the protein and solvent (if used)
protein_ff="amber14-all.xml" 
solvent_ff="amber14/tip3pfb.xml"
forcefield = app.ForceField(protein_ff, solvent_ff)

# Generate and add the forcefeild terms to the "forcefeild" function holdiding the protein and solver terms
#gaff = GAFFTemplateGenerator(molecules=Molecule.from_rdkit(rdkit_ligand, allow_undefined_stereo=True))
gaff = GAFFTemplateGenerator(molecules=off_ligand)


forcefield.registerTemplateGenerator(gaff.generator)

# The modeller collects together the molecular data (positions and toplology) ready for combining with the forcefeild
modeller = app.Modeller(complex_topology, complex_positions)

#Solvate the complex
modeller.addSolvent(forcefield, padding=0.15*unit.nanometers)

Exception ignored in: <function DcdTrajectory.__del__ at 0x7ff7c9da95a0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/simpletraj/trajectory.py", line 359, in __del__
    if self.dcd:
AttributeError: 'DcdTrajectory' object has no attribute 'dcd'
Exception ignored in: <function DCDReader.__del__ at 0x7ff7c9d67be0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/simpletraj/dcd/dcd.py", line 81, in __del__
    self.close()
  File "/opt/conda/lib/python3.10/site-packages/simpletraj/dcd/dcd.py", line 77, in close
    self._finish_dcd_read()
  File "/opt/conda/lib/python3.10/site-packages/simpletraj/dcd/dcd.py", line 101, in <lambda>
    DCDReader._finish_dcd_read = lambda self: _dcdmodule._finish_dcd_read( self )
AttributeError: _dcd_C_ptr is not an attribute1


# Running the simulation

We have now created a unified set of coordinates and a unified topology connecting all those coordinates and a forcefield with all the necessary terms for describing the physics between the atoms and molecules. We will now set the physical conditions for the simulation, such as the temperature and timestep for the simulation to run.

The simulation below will take a little while to set up and then the number of steps will progress quite rapidly, depending on the number of atoms in the system and how often we checkpoint the simulation.

We have selected the following parameters below for the simulation.

* Integrator : Langevin
* Nonbonded cutoff distance : 1 Nanometer
* friction coefficient : 0.1 picosecond
* Temperature : 300 kelvin
* Timestep : 0.004 picoseconds
* Number of timesteps : 2000 steps
* Total simulation time : 8 picoseconds
* Number of stpes between checkpoints : 100 steps
* Real world time between steps : 0.4 picoseconds
* Water box to generate 0.15 nanometers.

# With explicit waters this will take a LONG TIME

In [133]:
# Uncomment the blow line to use GPU accelleration
#platform = Platform.getPlatformByName('CUDA')

# setting of the chemical system
system = forcefield.createSystem(modeller.topology, nonbondedMethod=NoCutoff,
        nonbondedCutoff=1*nanometer, constraints=HBonds)

# settings for how bit the timestep should be
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

# Collect everything together to make a simulation instance
simulation = Simulation(modeller.topology, system, integrator)

# Set starting positions
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()

# Add a PDB reporter to the simulation
simulation.reporters.append(PDBReporter('assets/cookbook/hydrated_topolgy.pdb', 1))


# Run one step to write the first frame
simulation.step(1)

# Remove the PDB reporter
simulation.reporters = [reporter for reporter in simulation.reporters if not isinstance(reporter, PDBReporter)]


# File location to save output and how often to save
simulation.reporters.append(DCDReporter('assets/cookbook/first_output.dcd', 25))
simulation.reporters.append(XTCReporter('assets/cookbook/first_output.xtc', 25))


# Report the physical properties
simulation.reporters.append(StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True))

# # Number of steps to run
simulation.step(40000)



#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-215540.94604492188,99.79475015273518
200,-208197.52890014648,166.12127896710012
300,-203259.83862304688,210.6017193304774
400,-199701.3759765625,237.43063481731338
500,-196849.77285766602,255.96887196690793
600,-194965.4187927246,267.32048185880484
700,-193886.92568969727,275.2545677071237
800,-192998.12063598633,284.2814224798359
900,-191763.25553894043,287.53696526942474
1000,-191582.20904541016,298.31212554041326
1100,-190672.40115356445,293.6730351893583
1200,-190387.88177490234,293.7341967395436
1300,-190649.07885742188,302.2164658126373
1400,-190024.25994873047,296.9989948331079
1500,-190229.6150817871,293.3146080560549
1600,-190743.07405090332,302.05554659667854
1700,-190200.56015014648,299.06440196601136
1800,-189969.95793151855,302.06556224367273
1900,-189969.98640441895,300.85755512940983
2000,-189486.78408813477,299.2269824214353
2100,-190650.81622314453,305.15040843557375
2200,-190151.4721069336,302.15615480719896


# Analysing the output

Notice that every 100 steps that we get a report on the state of the system". It takes around 1000 steps to reach our target temperature.

In [135]:
import mdtraj as md
import nglview as nv

import MDAnalysis as mda


traj = nv.SimpletrajTrajectory("assets/cookbook/first_output.xtc", "assets/cookbook/hydrated_topolgy.pdb")
print(f"Trajectory has {traj.n_frames} frames")
viewtraj = nv.show_simpletraj(traj)
viewtraj.add_unitcell()
viewtraj.clear_representations()
viewtraj.add_representation("ribbon",color_scheme="residueindex")
viewtraj.add_ball_and_stick("ligand")
viewtraj 

Trajectory has 1600 frames


NGLWidget(max_frame=1599)